In [1]:
!pwd

/Users/miles/proj/tripshark/nb


In [2]:
!wc ../data/1_gtfs/routes.txt

     227    1363   23818 ../data/1_gtfs/routes.txt


In [4]:
import pandas as pd

In [14]:
import os

In [15]:
gtfs_files = os.listdir('../data/1_gtfs')

In [26]:
gtfs = {filename.split('.')[0]: pd.read_csv(
        os.path.join('../data/1_gtfs', filename)) for filename in gtfs_files}

In [27]:
for table in gtfs_tables:
    print(table + '\n' + '-'*len(table))
    print(gtfs[table].head())
    print('\n')

fare_rules
----------
   fare_id  route_id  origin_id  destination_id
0      101  100001.0        NaN             NaN
1      110  100001.0        NaN             NaN
2      101  100002.0        NaN             NaN
3      110  100002.0        NaN             NaN
4      102  100003.0        NaN             NaN


agency
------
  agency_id                agency_name  \
0         1              Metro Transit   
1        23            City of Seattle   
2        40              Sound Transit   
3       KMD  Kingcounty Marine Divison   

                                          agency_url      agency_timezone  \
0                        http://metro.kingcounty.gov  America/Los_Angeles   
1                             http://www.seattle.gov  America/Los_Angeles   
2                        http://www.soundtransit.org  America/Los_Angeles   
3  http://www.kingcounty.gov/transportation/kcdot...  America/Los_Angeles   

  agency_lang    agency_phone  \
0          EN    206-553-3000   
1          

In [30]:
gtfs['trips'].shape

(52194, 10)

In [31]:
gtfs['trips'].head()

,route_id,trip_id,service_id,trip_short_name,trip_headsign,route_short_name,direction_id,block_id,shape_id,fare_id
0,100173,18155895,2132,LOCAL,MADRONA AND 34TH AVE VIA E CHERRY ST,NaN,0,3992664,21003564,101
1,100173,18155899,2132,LOCAL,MADRONA AND 34TH AVE VIA E CHERRY ST,NaN,0,3992678,21003564,101
2,100173,18155902,2132,LOCAL,MADRONA AND 34TH AVE VIA E CHERRY ST,NaN,0,3992664,21003564,101
3,100173,18155906,2132,LOCAL,MADRONA AND 34TH AVE VIA E CHERRY ST,NaN,0,3992644,21003562,101
4,100173,18155908,2132,LOCAL,MADRONA AND 34TH AVE VIA E CHERRY ST,NaN,0,3992678,21003564,101


In [32]:
gtfs['shapes'].head()

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled
0,10002552,2,47.612152,-122.281746,0.0
1,10002552,3,47.612160,-122.281761,0.0
2,10002552,4,47.612164,-122.281830,16.6
3,10002552,5,47.612122,-122.281921,45.2
4,10002552,6,47.612202,-122.282005,80.5


In [34]:
import json

In [54]:
top_shapes = gtfs['trips'].groupby('shape_id').count()['trip_id'].sort_values(ascending=False).head(10)
tsi = top_shapes.index
set(tsi)

{10036407,
 10599537,
 11098501,
 20044004,
 20098504,
 20675007,
 21036411,
 21599536,
 30672002,
 41672006}

In [58]:
tr = gtfs['trips'][gtfs['trips']['shape_id'].isin(tsi)]['route_id'].unique()

In [59]:
gtfs['routes'][gtfs['routes']['route_id'].isin(tr)]

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_url
110,1,100210,36,NaN,3,Othello Stn - Beacon Hill - Downtown Sea,http://metro.kingcounty.gov/schedules/036/n0.html
118,1,100224,44,NaN,3,Ballard - Montlake,http://metro.kingcounty.gov/schedules/044/n0.html
190,1,102548,B Line,B-Line Rapid Ride,3,B-Line Rapid Ride,http://metro.kingcounty.gov/travel-options/bus...
204,1,102615,E Line,E-Line Rapid Ride,3,Aurora Village to Downtown Seattle,http://metro.kingcounty.gov/travel-options/bus...
212,23,100340,Stcr SLU,NaN,0,South Lake Union Streetcar,http://www.seattlestreetcar.org/slu.htm
221,40,100479,LINK,Link light rail,0,LINK to Sea-Tac,http://www.soundtransit.org/Schedules/ST-Expre...


In [60]:
gtfs['shapes'].head()

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled
0,10002552,2,47.612152,-122.281746,0.0
1,10002552,3,47.612160,-122.281761,0.0
2,10002552,4,47.612164,-122.281830,16.6
3,10002552,5,47.612122,-122.281921,45.2
4,10002552,6,47.612202,-122.282005,80.5


In [128]:
import geojson
from geojson import Feature, Point, LineString, FeatureCollection

def get_shape(shape_id, gtfs=gtfs):
    """Return a shape sorted by sequence number."""
    shape = gtfs['shapes'][gtfs['shapes']['shape_id'] == shape_id].sort_values(by='shape_pt_sequence')
    return shape
    
def shape_to_geojson(shape_id, gtfs=gtfs):
    """Return GeoJSON string for the given shape ID."""
    shape = get_shape(shape_id)
    shape.columns = ['id', 'seq', 'lat', 'lon', 'dist']  # Rename columns for brevity
    ls = LineString([(p[1]['lon'], p[1]['lat']) for p in shape.iterrows()])
    return geojson.dumps(Feature(geometry=ls, id=shape_id, className={'baseVal': 'bus_route'}))

In [129]:
gj = shape_to_geojson(10002552)

In [130]:
with open('../webapp/route.geojson', 'w') as f_out:
    f_out.write(gj)

In [125]:
shape_id = 10002552
shape = get_shape(shape_id)
shape.columns = ['id', 'seq', 'lat', 'lon', 'dist']  # Rename columns for brevity
ls = LineString([(p[1]['lon'], p[1]['lat']) for p in shape.iterrows()])
fc = FeatureCollection([Feature(geometry=ls, id=shape_id, className={'baseVal': 'bus_route'})])


In [126]:
geojson.dumps(fc)

'{"type": "FeatureCollection", "features": [{"className": {"baseVal": "bus_route"}, "geometry": {"type": "LineString", "coordinates": [[-122.281746, 47.612152000000002], [-122.28176100000002, 47.612159999999996], [-122.28183, 47.612164], [-122.281921, 47.612121999999999], [-122.282005, 47.612202000000003], [-122.28207399999999, 47.612278000000003], [-122.282471, 47.613246999999994], [-122.28253899999999, 47.613419], [-122.282646, 47.613582999999998], [-122.28280600000001, 47.613765999999998], [-122.28387499999999, 47.615184999999997], [-122.28391299999998, 47.615237999999998], [-122.28461499999999, 47.616168999999999], [-122.28461499999999, 47.616459000000006], [-122.284599, 47.616692], [-122.28456899999999, 47.617618999999998], [-122.28472099999999, 47.617885999999999], [-122.28482099999999, 47.618000000000002], [-122.284927, 47.618103000000005], [-122.28506499999999, 47.618190999999996], [-122.285225, 47.618262999999999], [-122.28529399999999, 47.618282000000001], [-122.285408, 47.61

In [117]:
shape

,id,seq,lat,lon,dist
0,10002552,2,47.612152,-122.281746,0.0
1,10002552,3,47.612160,-122.281761,0.0
2,10002552,4,47.612164,-122.281830,16.6
3,10002552,5,47.612122,-122.281921,45.2
4,10002552,6,47.612202,-122.282005,80.5
5,10002552,7,47.612278,-122.282074,113.5
6,10002552,8,47.613247,-122.282471,479.2
7,10002552,9,47.613419,-122.282539,545.0
8,10002552,10,47.613583,-122.282646,611.2
9,10002552,11,47.613766,-122.282806,689.4
